In [2]:
import requests  # Biblioteca padrão para fazer chamadas HTTP (GET) na web
import json      # Biblioteca para manipular o formato de texto JSON
from json import dump, dumps
from hdfs import InsecureClient # Cliente para conectar no Hadoop

# 1. Configuração do Cliente HDFS
client = InsecureClient('http://namenode:9870', user='root')

# É uma boa prática definir um cabeçalho (header) com uma chave de API.
# Para este exemplo, a API funciona sem chave, mas em produção, você deve usar uma.
headers = {
    "Content-Type": "application/json"
}

# Função para cada url
def ingestao_bronze(url_api, nome_arquivo_destino):
    print(f"Baixando dados de: {nome_arquivo_destino}...")
    
    # 2. Extração (Extract)
    response = requests.get(url_api, headers=headers)
    
    dados_json = response.json()

    caminho_completo = f'/datalake/bronze/{nome_arquivo_destino}.json'
    
    # 3. Carga no HDFS (Load)
    with client.write(caminho_completo, encoding='utf-8', overwrite=True) as writer:
        dump(dados_json, writer)
        
    print(f"✅ Arquivo salvo no HDFS: {caminho_completo}")

# Executando a função
URL_SUICIDIO = "https://ghoapi.azureedge.net/api/SDGSUICIDE?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY' and Dim2 eq 'AGEGROUP_YEARSALL'"
URL_DEPRESSAO = "https://ghoapi.azureedge.net/api/MH_12?$filter=TimeDim eq 2021 and SpatialDimType eq 'COUNTRY'"

ingestao_bronze(URL_SUICIDIO, "raw_suicidio")
ingestao_bronze(URL_DEPRESSAO, "raw_depressao")

Baixando dados de: raw_suicidio...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_suicidio.json
Baixando dados de: raw_depressao...
✅ Arquivo salvo no HDFS: /datalake/bronze/raw_depressao.json
